# 네이버 뉴스 API 연결해서 뉴스 스크래핑
[참고 자료]
* https://developers.naver.com/docs/serviceapi/search/blog/blog.md#%EA%B2%80%EC%83%89-api-%EB%B8%94%EB%A1%9C%EA%B7%B8-%EA%B2%80%EC%83%89-%EA%B5%AC%ED%98%84-%EC%98%88%EC%A0%9C
* https://contents.premium.naver.com/chatgpt/buff/contents/231204223117640hg


In [3]:
MY_CLIENT_ID = "hZm0ny7bv69wAHcdYffq"
MY_CLIENT_SECRET = "pIyAPPtzD2"

In [25]:
import urllib.request
import pandas as pd
import json
import re
import html

# 네이버 API의 클라이언트 ID와 시크릿 키 설정
CLIENT_ID = MY_CLIENT_ID  # 실제 클라이언트 ID를 입력하세요
CLIENT_SECRET = MY_CLIENT_SECRET  # 실제 클라이언트 시크릿을 입력하세요

# 검색할 키워드 목록
SEARCH_QUERIES = ["주식", "채권", "금융"]

# API 호출 시 설정
DISPLAY_COUNT = 100  # 한 페이지에 표시할 뉴스 항목 수
START_INDEX = 1  # 검색 결과의 시작 인덱스
END_INDEX = 300  # 검색 결과의 종료 인덱스
SORT_ORDER = "sim"  # 정렬 방식: "sim" (유사도) 또는 "date" (날짜)

# 뉴스 데이터를 저장할 데이터프레임 초기화
news_df = pd.DataFrame(columns=["Category", "Title", "Original Link", "Publication Date"])

def fetch_news(query):
    """
    주어진 검색어에 대해 뉴스 데이터를 가져와서 데이터프레임에 추가합니다.
    
    :param query: 검색어
    """
    global news_df
    encoded_query = urllib.parse.quote(query)  # 검색어를 URL 인코딩
    current_index = len(news_df)  # 현재 데이터프레임의 인덱스
    
    for start_index in range(START_INDEX, END_INDEX, DISPLAY_COUNT):
        url = f"https://openapi.naver.com/v1/search/news?query={encoded_query}&display={DISPLAY_COUNT}&start={start_index}&sort={SORT_ORDER}"
        
        try:
            request = urllib.request.Request(url)
            request.add_header("X-Naver-Client-Id", CLIENT_ID)
            request.add_header("X-Naver-Client-Secret", CLIENT_SECRET)
            response = urllib.request.urlopen(request)
            response_code = response.getcode()
            
            if response_code == 200:
                response_body = response.read()
                response_dict = json.loads(response_body.decode('utf-8'))
                items = response_dict['items']
                
                for item in items:
                    # HTML 태그 제거
                    clean_title = re.sub(re.compile('<.*?>'), '', item['title'])
                    clean_pub_date = re.sub(re.compile('<.*?>'), '', item['pubDate'])
                    
                    # 데이터프레임에 뉴스 항목 추가
                    news_df.loc[current_index] = [
                        query,  # 카테고리 추가
                        html.unescape(clean_title),  # HTML 엔티티 제거
                        html.unescape(item['originallink']),  # HTML 엔티티 제거
                        clean_pub_date
                    ]
                    current_index += 1
            else:
                print(f"Error Code: {response_code}")
                error_details = response.read().decode('utf-8')
                print(f"Error Details: {error_details}")

        except urllib.error.HTTPError as e:
            print(f"HTTPError: {e.code} - {e.reason}")
            error_details = e.read().decode('utf-8')
            print(f"Error Details: {error_details}")

# 각 검색어에 대해 뉴스 데이터 가져오기
for query in SEARCH_QUERIES:
    fetch_news(query)

# 데이터프레임을 CSV 파일로 저장
news_df.to_csv('news_data.csv', index=False)
print("CSV 파일이 'news_data.csv'로 저장되었습니다.")

# 데이터프레임 확인
news_df

CSV 파일이 'news_data.csv'로 저장되었습니다.


,Category,Title,Original Link,Publication Date
0,주식,'부동산보단 주식'…투자 전환 강조한 김병환 금융위원장,https://www.hankyung.com/article/2024081288721,"Mon, 12 Aug 2024 17:51:00 +0900"
1,주식,"국민연금 ""핀란드 주식투자 배당 세금 96억원 환급 소송 승소""",https://www.yna.co.kr/view/AKR2024081205530053...,"Mon, 12 Aug 2024 10:43:00 +0900"
2,주식,"토스증권, ‘주식모으기’ 서비스 수수료 무료화 선언",https://view.asiae.co.kr/article/2024081209554...,"Mon, 12 Aug 2024 09:55:00 +0900"
3,주식,"토스증권 ""주식모으기 이제 무료로 이용하세요""",https://www.dt.co.kr/contents.html?article_no=...,"Mon, 12 Aug 2024 09:26:00 +0900"
4,주식,"토스證, '주식모으기' 서비스 거래 수수료 안 받는다",https://biz.sbs.co.kr/article_hub/20000186523?...,"Mon, 12 Aug 2024 09:54:00 +0900"
...,...,...,...,...
895,금융,"농협상호금융, ‘쌀맛 나는 하루’ 캠페인 전개",https://www.asiatoday.co.kr/view.php?key=20240...,"Mon, 12 Aug 2024 17:44:00 +0900"
896,금융,"대규모 부정대출 우리금융, 이제 와 '무관용 원칙' 공표",https://www.kbanker.co.kr/news/articleView.htm...,"Mon, 12 Aug 2024 14:12:00 +0900"
897,금융,"하나금융, 어르신 보양식 나눔 봉사활동 실시",http://www.babytimes.co.kr/news/articleView.ht...,"Mon, 12 Aug 2024 09:34:00 +0900"
898,금융,"하나금융, 혹서기 취약한 어르신들 위한 봉사활동 진행",http://www.thefirstmedia.net/news/articleView....,"Mon, 12 Aug 2024 15:44:00 +0900"


In [26]:
# 카테고리별 데이터 개수 확인
category_counts = news_df.groupby("Category").size()
print("카테고리별 뉴스 데이터 개수:")
print(category_counts)

카테고리별 뉴스 데이터 개수:
Category
금융    300
주식    300
채권    300
dtype: int64


In [31]:
news_df['Title'][0]

"'부동산보단 주식'…투자 전환 강조한 김병환 금융위원장"